## This notebook converts the classifications of Faster-RCNN into the tracking format.


In [2]:
%pylab inline
import os
import glob
import sys
import tifffile

import det_sort as ds
import det_helper as dh
sys.path.append("/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/Faster-RCNN-TensorFlow-Python3.5/")
import det_process_faster as dpf
import tensorflow as tf
tf.reset_default_graph()
sys.path.append("/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/darknet/")
from darknet_access import run_command

from importlib import reload
reload(dpf)


base_path = "/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/"
#input model path
tfmodel = base_path+"tracking/vgg16_faster_rcnn_iter_20000.ckpt"
#input file path
path_to_folder = base_path+'acquisitions/zstacks/Pos3_perox.ome.tif/'
file_name = "Pos3_perox.ome.tif_MMStack_Pos0.ome.tif"
class_to_track = 'cell - hela peroxisome all'
#class_to_track = 'cell - erythroblast dapi'
class_to_track = 'cell - neuroblastoma phalloidin'
#folder locations for intermediate and output files.
output_figs_folder = base_path+'output_figs/'
isolated_cell_folder = base_path+"raw_imgs/"
output_stacks_folder = base_path+"reconstructed_stacks/"

#Create the folders if they don't exist.
os.makedirs(output_figs_folder, exist_ok=True)
os.makedirs(isolated_cell_folder, exist_ok=True)
os.makedirs(output_stacks_folder, exist_ok=True)

#Display output images?
display = False


### CODE starts:
#dpf.start_import(path_to_folder+file_name,'global',tfmodel)


        
xpos,ypos,zpos,number_of_images = dh.read_meta_data_from_micro_manager_stack(path_to_folder+file_name)

if not os.path.isfile(tfmodel + '.meta'):
    raise IOError(('{:s} not found.\nDid you download the proper networks from '
                   'our server and place them properly?').format(tfmodel + '.meta'))

colours = np.random.rand(32,3) #used only for display

mot_tracker = []
mot_tracker = ds.Sort()
d_z = dict()

tgt = {}
imgs = {}
im_stack = tifffile.TiffFile(path_to_folder+file_name).asarray()

for i in range(0,number_of_images):
    fname = '%d.txt'%i
    
    if xpos != []:
        x = xpos[i]
        y = ypos[i]
        z = zpos[i]

        #p,c,d = dh.convert_Faster_RCNN_output(path_to_folder+file_name[:-4]+"/"+fname,class_to_track)
        darknet_path = "/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/darknet/results/"
        p,c,d = dh.convert_Faster_RCNN_output(darknet_path+'file'+fname,class_to_track)
        
        if p != []:
            print( fname)
            dets = []
            for c0,d0,p0 in zip(c,d,p):

                detstxt = [str(i),"-1",c0[0],c0[1],c0[2]-c0[0],c0[3]-c0[1]]
                
                #drt+=1
                dets.append(np.array(detstxt[2:]).astype(np.float64))
            dets = np.array(dets)

            dets[:,2:4] += dets[:,0:2]


            #The first detection is used as the centre of the 
            if dets.size !=0:
                xp0 = int(dets[0,-3])
                yp0 = int(dets[0,-2])
                zp0 = float(dets[0,-1])


            im_gray = im_stack[i,:,:]
            im = np.zeros((im_gray.shape))
            im[:,:] = im_gray
            im = im.astype(np.float32)/16.0
            im = im[::2,::2]
            imgs[i] = im
            
            if(display):
                fig = figure()
                ax1 = fig.add_subplot(111, aspect='equal')
                imt = im
                imt = (imt/np.max(imt))*255.0
                ax1.imshow(imt,cmap="Greys_r")
                

            #Updates tracker with the latest detection.
            trackers = mot_tracker.update(dets)

            for d in trackers:
                d = d.astype(np.int32)

                if(display):
                    ax1.add_patch(Rectangle((d[0],d[1]),d[2]-d[0],d[3]-d[1],fill=False,lw=3,ec='blue'))
                    ax1.set_adjustable('box-forced')
                if d[-1] not in tgt:
                     tgt[d[-1]] = []
               
                #print (d)

            #scan through the targets.    
            for dt in tgt:
                fl = False
                for d in trackers:
                    if d[-1] == dt:
                        tgt[dt].append([d,z,i])
                        #save_out = tifffile.TiffWriter('Stack'+str(d[-1])+"_"+str(z)+"_"+str(round(d[0],2))+"_"+str(round(d[1],2))+'.tif')
                      .  #save_out.save(im[int(d[1]):int(d[3]),int(d[0]):int(d[2])])
                        fl = True
                if fl == False:
                    tgt[dt].append([[],z,i])

#Trims off the [] at the end of the sequences.  
for tg in tgt:
    ab =tgt[tg]

    proceed = False
    c = ab.__len__()
    for b in reversed(ab):

        c = c-1
        if b[0] == [] and proceed == False:
            continue   
        if b[0] != []:
            proceed = True
        if b[0] == [] and proceed == True:
            ab[c][0] = ab[c+1][0]

    tgt[tg] = ab




#Now we save out. The data is now ordered in relation to track. 
files = glob.glob(isolated_cell_folder +'/*')
for f in files:
    os.remove(f)

    
for dt in tgt:
    track = tgt[dt]
    for d,z,i in track:
        if d != []:
            try:
                if d[1]<0:
                    d[1]= 0
                save_out = tifffile.TiffWriter(isolated_cell_folder +'/Stack'+str(d[-1])+"_"+str(z)+"_"+str(floor(d[0]))+"_"+str(floor(d[1]))+'.tif')
                save_out.save(imgs[i][int(d[1]):int(d[3]),int(d[0]):int(d[2])])
            except:
                pass
            

#Finally we add in the tracks into visuals: optional.
if(display):
    for i in range(0,number_of_images):
        im_gray = im_stack[i,:,:]

        im = im_gray
        im = im.astype(np.float32)
        im = (im/np.max(im))*255.0
        im = im.astype(np.uint8)
        im = im[::2,::2]

        fig = figure()
        ax1 = fig.add_subplot(111, aspect='equal')
        ax1.imshow(im,cmap="Greys_r")
        for dt in tgt:
            track = tgt[dt]
            for d,z,it in track:
                if it == i and d != []:
                    #ax1 = fig.add_subplot(111, aspect='equal')
                    ax1.add_patch(Rectangle((d[0],d[1]),d[2]-d[0],d[3]-d[1],fill=False,lw=3,ec=colours[int(d[4]%32),:]))
                    #ax1.set_adjustable('box-forced')
                    #ax1.axes.get_xaxis().set_visible(False)
                    #ax1.axes.get_yaxis().set_visible(False)
        
        ax1.set_axis_off()
        fig.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0,hspace = 0, wspace = 0)
        ax1.margins(0,0)
        ax1.xaxis.set_major_locator(NullLocator())
        ax1.yaxis.set_major_locator(NullLocator())
        fig.savefig(output_figs_folder+'imgs'+str(i)+'.png',dpi=100,bbox_inches='tight',pad_inches=0)

#Removes temporary files from folder.
files = glob.glob(output_stacks_folder+'/Stack*')
for f in files:
    os.remove(f)
#Generates stacks from individual images.
dh.sort_imgs_into_stacks(isolated_cell_folder, output_stacks_folder)


Populating the interactive namespace from numpy and matplotlib


/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


UnboundLocalError: local variable 'number_of_images' referenced before assignment

In [ ]:
import xml.etree.ElementTree as ET
import sys
import _pickle as pickle
sys.path.append('darknet/scripts/')
%pylab inline
def parse_rec(filename):
    """ Parse a PASCAL VOC xml file """
    tree = ET.parse(filename)
    objects = []
    for obj in tree.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        #obj_struct['pose'] = obj.find('pose').text
        #obj_struct['truncated'] = int(obj.find('truncated').text)
        obj_struct['difficult'] = int(obj.find('difficult').text)
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text),
                              int(bbox.find('ymin').text),
                              int(bbox.find('xmax').text),
                              int(bbox.find('ymax').text)]
        objects.append(obj_struct)

    return objects
def parse_test_set(test_file):
    f = open(test_file,'r')
    out = []
    for line in f:
        out.append(line.strip('\n'))
    return out
import sys
sys.path.append("/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/darknet/scripts/")
import subprocess

path_data = "/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/Faster-RCNN-TensorFlow-Python3.5/data/global/2018/"

from subprocess import call
darknet_path = "/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/darknet/"
image_path = "/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/Faster-RCNN-TensorFlow-Python3.5/data/hela_peroxisome_all_class/2018/JPEGImages/"
sting2 = darknet_path+"darknet detector test "+path_data+"obj_global_class_test.data "+darknet_path+"cfg/yolo-obj.cfg "+darknet_path+"model/yolo-obj_10000.weights "+image_path+"107592.jpg"

#sting2 = "ls -l"
p = subprocess.Popen("\""+sting2+"\"", shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line)
retval = p.wait()

In [ ]:
sting3 ="/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/darknet/darknet detector test /Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/Faster-RCNN-TensorFlow-Python3.5/data/global/2018/obj_global_class_test.data /Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/darknet/cfg/yolo-obj.cfg /Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/darknet/model/yolo-obj_10000.weights /Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/Faster-RCNN-TensorFlow-Python3.5/data/hela_peroxisome_all_class/2018/JPEGImages/107592.jpg"


In [ ]:
import subprocess
p = subprocess.Popen(sting4, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line)
retval = p.wait()

In [ ]:
print( subprocess.check_output([sting4]))

In [ ]:
print(subprocess.check_output(sting4))

In [ ]:
sting4 = ["/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/darknet/darknet", "detector","test /Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/Faster-RCNN-TensorFlow-Python3.5/data/global/2018/obj_global_class_test.data /Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/darknet/cfg/yolo-obj.cfg /Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/darknet/model/yolo-obj_10000.weights /Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/Faster-RCNN-TensorFlow-Python3.5/data/hela_peroxisome_all_class/2018/JPEGImages/107592.jpg"]



In [ ]:
process = subprocess.Popen(["/bin/bash"], shell=False, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
process.stdin.write('echo it works!\n'.encode())
for line in process.stdout.readlines():
    print (line)
process.stdin.write('date\n'.encode())
process.stdout.readline()



In [ ]:
import subprocess
process = subprocess.Popen(sting4, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
stdout = process.communicate()
print ('STDOUT:{}'.format(stdout))

In [ ]:
import shlex
def run_command(command):
    process = subprocess.Popen(shlex.split(command), stdout=subprocess.PIPE)
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print (output.strip())
    rc = process.poll()
    return rc

In [ ]:
run_command(sting3)

In [ ]:
shlex.split(sting3)

In [ ]:
import sys
sys.path.append('/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/darknet')
import test as it

In [ ]:
sting3 ="/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/darknet/darknet detector test /Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/Faster-RCNN-TensorFlow-Python3.5/data/global/2018/obj_global_class_test.data /Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/darknet/cfg/yolo-obj.cfg /Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/darknet/model/yolo-obj_10000.weights /Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/Faster-RCNN-TensorFlow-Python3.5/data/hela_peroxisome_all_class/2018/JPEGImages/107592.jpg"

it.run_command(sting3)


In [2]:
import tifffile as tifffile
import numpy as np

img_stack = {}
img_stack[0] = np.zeros((50,50))
img_stack[1] = np.zeros((50,50))
img_stack[1][10:40,10:40] = 255.0
img_stack[2] = np.zeros((50,50))
img_stack[2][10:40,10:40] = 255.0
img_stack[2][20:30,20:30] = 0.0
img_stack[3] = np.zeros((50,50))
img_stack[3][10:40,10:40] = 255.0
img_stack[3][24:26,24:26] = 0.0
img_stack[4] = np.zeros((50,50))
img_stack[4][10:40,10:40] = 255.0
img_stack[5] = np.zeros((50,50))
im_stk = np.array(img_stack)
micromanager_metadata = {'image_description':'jajajaj'}

tif_stack = tifffile.TiffWriter("/Users/dwaithe/Desktop/out.tiff",imagej=True)
out_img = []
for txt in range(0,5):
    #print file
    out_img.append(img_stack[txt].astype(np.float32))

tif_stack.save(np.array(out_img).astype(np.float32))




In [3]:
im_stk = np.zeros((6,500,500)).astype(np.float16)

im_stk[2,10:40,10:40] = 255.0
im_stk[3,10:40,10:40] = 255.0
im_stk[4,10:40,10:40] = 255.0

tifffile.imsave("/Users/dwaithe/Desktop/out4.tiff",im_stk,description="jaja")

In [47]:
bytes = 
print(bytes)

b'\x80\x00\x00\x00'


In [44]:
 # Python 3
key = bytearray(100000)
key[30] = 73
print (key)

bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00I\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [95]:
for it in range(6,20):
    print(ord(overlay[1][it:it+1].decode()))

1
0
0
29
0
75
0
69
0


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 0: invalid start byte

In [ ]:
write(im_stk, pathname, type='uint16', verbose=True)

xMax 500 yMax 500
size 128
C 0 Z 0 T 0 ol 0 imageSize 0
Iout
227
rtype 1
153 156 296 326
16
x1y1x2y2 0.0 0.0 0.0 0.0
stroke_width 5
stroke_col 255 255 0 0
fillcolour 0 0 0 0
0
options 32
arrow_style 0
arrow_head 0
rounded_rect 0
position 2
name 






xMax 500 yMax 500
size 128
C 0 Z 1 T 1 ol 0 imageSize 0
Iout
227
rtype 1
153 156 296 326
16
x1y1x2y2 0.0 0.0 0.0 0.0
stroke_width 5
stroke_col 255 255 0 0
fillcolour 0 0 0 0
0
options 0
arrow_style 0
arrow_head 0
rounded_rect 0
position 0
name 






xMax 500 yMax 500
size 128
C 0 Z 2 T 2 ol 0 imageSize 0
Iout
227
rtype 1
155 159 290 314
16
x1y1x2y2 0.0 0.0 0.0 0.0
stroke_width 5
stroke_col 255 255 0 0
fillcolour 0 0 0 0
0
options 0
arrow_style 0
arrow_head 0
rounded_rect 0
position 0
name 






ROI
xMax 500 yMax 500
size 128
C 0 Z 0 T 0 ol 0 imageSize 0
Iout
227
rtype 1
155 159 290 314
16
x1y1x2y2 0.0 0.0 0.0 0.0
stroke_width 0
stroke_col 0 0 0 0
fillcolour 0 0 0 0
0
options 0
arrow_style 0
arrow_head 0
rounded_rect 0
position 0
name 



In [30]:
import lxml.etree
import tifffile
overlay = [b'Iout\x00\xe3\x01\x00\x00U\x006\x00y\x00Z\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x0e\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000\x000\x000\x001\x00-\x000\x001\x000\x003\x00-\x000\x000\x007\x002', b'Iout\x00\xe3\x01\x00\x00\x1d\x00K\x00E\x00\x8b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000\x000\x000\x001\x00-\x000\x000\x004\x009\x00-\x000\x001\x000\x007']

pathname ="/Users/dwaithe/Desktop/out.tiff"
kwargs ={'ijmetadata':{'slices': 27, 'images': 27, 'ImageJ': '1.52g', 'Overlays':overlay , 'loop': False}}

In [14]:
#int(255 255 0 0)
num = 4294901760
byte = (num).to_bytes(4, 'big')

fT = int.from_bytes(byte[0:1], byteorder='big')
fR = int.from_bytes(byte[1:2], byteorder='big')
fG = int.from_bytes(byte[2:3], byteorder='big')
fB = int.from_bytes(byte[3:4], byteorder='big')
bT = (fT).to_bytes(1, 'big')
bR = (fR).to_bytes(1, 'big')
bG = (fG).to_bytes(1, 'big')
bB = (fB).to_bytes(1, 'big')

print('colour',fT,fR,fG,fB)
print('byte',bT+bR+bG+bB)
print('col_nt',int.from_bytes(bT+bR+bG+bB, byteorder='big'))

colour 255 255 0 0
byte b'\xff\xff\x00\x00'
col_nt 4294901760


In [ ]:
from lxml import etree
root = etree.Element("root")
child1 = etree.SubElement(root, "child1")
child2 = etree.SubElement(root, "child2")
child1.set("Points", '5,4')
new_s = lxml.etree.tostring(root, encoding="UTF-8", xml_declaration=True)

In [383]:
tfile.imagej_metadata

{'ImageJ': '1.52g',
 'Info': 'ImageDescription: {"shape": [6, 500, 500]}\nSoftware: tifffile.py\nDateTime: 2018:10:31 15:45:15\n',
 'Overlays': [b'Iout\x00\xe3\x01\x00\x00\x99\x00\x9c\x01(\x01F\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\xa0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  b'Iout\x00\xe3\x01\x00\x00\x99\x00\x9c\x01(\x01F\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [285]:
roi

b"Iout\x00\xe3\x01\x00\x00\xb3\x00\x94\x015\x01'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00t\x00e\x00s\x00t\x001"

In [ ]:
raw_points = ((4,5),(3,4),(2,1),(4,8),(4,3))
z_count = 4

polygons = create_polygons_from_points(raw_points, z_count)